In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def parse_song(artist, song):
    a_list = []
    for i in artist:
        a = "%20".join(i.lower().split(" "))
        a_list.append(a)
    
    s = "%20".join(song.lower().split(" "))

    return a_list[0], s

In [9]:
def parse_lyrics(soup_object):
    """Function to find the lyrics from the soup object and clean it"""
    lyrics = soup_object.find(id="ctl00_ContentPlaceHolder1_lbllyrics_simple").text
    
    #process it a little
    lyrics = lyrics.split("[") #clean up the lyrics from format [time] lyrics [time] lyrics
    title = lyrics[0].split("]")[0] #get the title of the song
    print(f"Now playing {title}")
    lyrics = {s[0]:s[1] for s in [l.split("]") for l in lyrics[1:]]}
    lyrics = lyrics.items() #return it as a touple [(timestamp, lyric)]
    return lyrics

In [13]:
def query_song(artist, song):
    """
    Function to query the website for the results page given 
    artist and song when we cant find it right away
    """
    search = f"{artist}%20{song}"
    query_url = f"https://www.rentanadviser.com/subtitles/subtitles4songs.aspx?q={search}"
    r = requests.get(query_url)
    soup = BeautifulSoup(r.text, "html.parser")

    table = soup.find("table")
    rows = table.find_all('tr')
    output = []
    for row in rows:
        cols=row.find_all('td')
        cols=[x for x in cols]
        link = cols[0].a['href']
        link = f"https://www.rentanadviser.com/subtitles/{link}"
        if "official" in link.lower(): #only want the official version 
            print("OFFICIAL WAS FOUND",link)
            return cols,parse_lyrics(BeautifulSoup(requests.get(link).text, "html.parser"))
        else:
            output.append(link) #if we dont find an official we just append it
    
    # if we didnt find any  official we just return the first link through parse_lyrics
    print("OFFICIAL WAS NOT FOUND", output[0])
    return cols,parse_lyrics(BeautifulSoup(requests.get(output[0]).text, "html.parser"))
a,x= query_song("ed sheeran","shape of you")

OFFICIAL WAS FOUND https://www.rentanadviser.com/subtitles/getsubtitle.aspx?artist=Ed%20Sheeran&song=Shape%20Of%20You%20(Official%20Video)
Now playing Ed Sheeran - Shape Of You (Official Video) Lyrics (.LRC) 



In [18]:
type(a[0])

bs4.element.Tag

In [19]:
a

[<td><a href="getsubtitle.aspx?artist=Ed%20Sheeran&amp;song=Shape%20Of%20You%20(Official%20Video)" target="_blank"><img alt="subtitle icon" src="../popular-music-videos/images/ccsub.png"/> Ed Sheeran - Shape Of You (Official Video) </a> Lyrics &amp; Subtitles</td>]

In [54]:
b = a[0].find(text=True).lower()

In [61]:
a = set["ed","shape of you"]

TypeError: 'type' object is not subscriptable

In [58]:
a in b

TypeError: 'in <string>' requires string as left operand, not set